# 13: Merge Tier data with VRS data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
from shapely import Point, wkb, wkt

In [2]:
data_month_single = '7'
data_month_double = '07'

## Load data

In [3]:
# load tier trip data
tier_trips = pd.DataFrame(pd.read_parquet("../Data/12_tier_part2.parquet", engine="pyarrow"))
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,e-scooter,0 days 00:10:00,0.617222,617.221763,b'\x01\x01\x00\x00\x00\xc7\xf4\x84%\x1e\xa0\x1...,b'\x01\x01\x00\x00\x00\x8d\x08\xc6\xc1\xa5\xa3...,1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,e-scooter,0 days 00:15:00,0.699559,699.558708,"b""\x01\x01\x00\x00\x00'2s\x81\xcb\xa3\x1c@x*\x...","b""\x01\x01\x00\x00\x00*9'\xf6\xd0\x9e\x1c@\x8d...",NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1584.0,Bonn Ramersdorf,801621 ...
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,e-scooter,0 days 00:10:00,0.766417,766.417469,b'\x01\x01\x00\x00\x00\xfcPi\xc4\xcc\x9e\x1c@\...,b'\x01\x01\x00\x00\x00\xb3\x08\xc5V\xd0\xa4\x1...,1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,e-scooter,0 days 00:50:00,10.615185,10615.184930,b'\x01\x01\x00\x00\x00\xa8qo~\xc3\xa4\x1c@G\x8...,b'\x01\x01\x00\x00\x00M\xd7\x13]\x17N\x1c@:z\x...,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,b'\x01\x01\x00\x00\x00\t\xfbv\x12\x11N\x1c@R\n...,b'\x01\x01\x00\x00\x00\x8b\x8a8\x9dd{\x1c@\x1b...,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1501.0,Bonn Beuel Rathaus,801621 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,e-scooter,0 days 00:30:00,8.561807,8561.806757,b'\x01\x01\x00\x00\x00\x1d\xaa)\xc9:\xac\x1c@;...,b'\x01\x01\x00\x00\x00\x87\xf9\xf2\x02\xecc\x1...,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1162.0,Bonn Rosental,801621 ...
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,e-scooter,0 days 00:30:00,8.172812,8172.812192,b'\x01\x01\x00\x00\x00[\x0c\x1e\xa6}c\x1c@\x1a...,b'\x01\x01\x00\x00\x00@3\x88\x0f\xec\xa8\x1c@;...,1162.0,Bonn Rosental,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,e-scooter,0 days 00:05:00,0.115253,115.252520,b'\x01\x01\x00\x00\x00\x8b\x89\xcd\xc7\xb5\x91...,b'\x01\x01\x00\x00\x00g|_\\\xaa\x92\x1c@\xf3\x...,1585.0,Bonn Küdinghoven,801621 POLYGON ((7.157885983913106 50.7227...,1585.0,Bonn Küdinghoven,801621 ...
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,b'\x01\x01\x00\x00\x00 \xb5\x89\x93\xfb\x9d\x1...,b'\x01\x01\x00\x00\x00`\xe7\xa6\xcd8\x9d\x1c@|...,2304.0,Bonn Alte Schulstr.,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...


In [4]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier_trips["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier_trips["end_location"], crs=4326))

In [5]:
# drop geometry columns in wrong format
tier_trips = tier_trips.drop(columns=["start_location", "end_location"])

In [6]:
# replace geometry columns with correct format
tier_trips['start_location'] = point_start
tier_trips['end_location'] = point_end

In [7]:
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,start_location,end_location
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,e-scooter,0 days 00:10:00,0.617222,617.221763,1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.15637 50.72230),POINT (7.15981 50.71792)
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,e-scooter,0 days 00:15:00,0.699559,699.558708,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1584.0,Bonn Ramersdorf,801621 ...,POINT (7.15996 50.71801),POINT (7.15509 50.72203)
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,e-scooter,0 days 00:10:00,0.766417,766.417469,1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.15508 50.72208),POINT (7.16095 50.71844)
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,e-scooter,0 days 00:50:00,10.615185,10615.184930,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.16090 50.71841),POINT (7.07626 50.76291)
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1501.0,Bonn Beuel Rathaus,801621 ...,POINT (7.07624 50.76284),POINT (7.12050 50.73918)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,e-scooter,0 days 00:30:00,8.561807,8561.806757,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1162.0,Bonn Rosental,801621 ...,POINT (7.16819 50.71135),POINT (7.09758 50.74229)
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,e-scooter,0 days 00:30:00,8.172812,8172.812192,1162.0,Bonn Rosental,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.09716 50.74206),POINT (7.16496 50.71347)
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,e-scooter,0 days 00:05:00,0.115253,115.252520,1585.0,Bonn Küdinghoven,801621 POLYGON ((7.157885983913106 50.7227...,1585.0,Bonn Küdinghoven,801621 ...,POINT (7.14229 50.72758),POINT (7.14323 50.72803)
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,2304.0,Bonn Alte Schulstr.,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.15428 50.74919),POINT (7.15354 50.74152)


In [8]:
# transform start_location to geometry
tier_trips = gpd.GeoDataFrame(tier_trips, geometry='start_location')

In [9]:
# transform end_location to geometry
tier_trips = gpd.GeoDataFrame(tier_trips, geometry='end_location')

In [10]:
# set start_location back to GDF geometry
tier_trips = gpd.GeoDataFrame(tier_trips, geometry='start_location')

In [11]:
tier_trips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 354421 entries, 801621 to 4807604
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   tier_trips_id             354421 non-null  int64          
 1   tier_trips_start_time     354421 non-null  datetime64[ns] 
 2   tier_trips_end_time       354421 non-null  datetime64[ns] 
 3   vehicle type              354421 non-null  object         
 4   trip_duration             354421 non-null  timedelta64[ns]
 5   trip_distance_kilometers  354421 non-null  float64        
 6   trip_distance_meters      354421 non-null  float64        
 7   start_stop_id             169948 non-null  float64        
 8   start_stop_name           169948 non-null  object         
 9   start_buffer_zone         354421 non-null  object         
 10  end_stop_id               169380 non-null  float64        
 11  end_stop_name             169380 non-null  

In [12]:
tier_trips

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,start_location,end_location
801621,497085150,2023-01-02 16:10:00,2023-01-02 16:20:00,e-scooter,0 days 00:10:00,0.617222,617.221763,1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.15637 50.72230),POINT (7.15981 50.71792)
801629,497085158,2023-01-02 21:25:00,2023-01-02 21:40:00,e-scooter,0 days 00:15:00,0.699559,699.558708,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1584.0,Bonn Ramersdorf,801621 ...,POINT (7.15996 50.71801),POINT (7.15509 50.72203)
801631,497085160,2023-01-02 21:50:00,2023-01-02 22:00:00,e-scooter,0 days 00:10:00,0.766417,766.417469,1584.0,Bonn Ramersdorf,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.15508 50.72208),POINT (7.16095 50.71844)
801637,497085166,2023-01-03 06:50:00,2023-01-03 07:40:00,e-scooter,0 days 00:50:00,10.615185,10615.184930,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.16090 50.71841),POINT (7.07626 50.76291)
801643,497085172,2023-01-03 14:50:00,2023-01-03 15:15:00,e-scooter,0 days 00:25:00,5.571567,5571.567347,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1501.0,Bonn Beuel Rathaus,801621 ...,POINT (7.07624 50.76284),POINT (7.12050 50.73918)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807552,1006547193,2023-12-29 10:30:00,2023-12-29 11:00:00,e-scooter,0 days 00:30:00,8.561807,8561.806757,NaN,None,801621 POLYGON ((7.157885983913106 50.7227...,1162.0,Bonn Rosental,801621 ...,POINT (7.16819 50.71135),POINT (7.09758 50.74229)
4807558,1006547199,2023-12-29 11:35:00,2023-12-29 12:05:00,e-scooter,0 days 00:30:00,8.172812,8172.812192,1162.0,Bonn Rosental,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.09716 50.74206),POINT (7.16496 50.71347)
4807582,1006547223,2023-12-28 15:15:00,2023-12-28 15:20:00,e-scooter,0 days 00:05:00,0.115253,115.252520,1585.0,Bonn Küdinghoven,801621 POLYGON ((7.157885983913106 50.7227...,1585.0,Bonn Küdinghoven,801621 ...,POINT (7.14229 50.72758),POINT (7.14323 50.72803)
4807597,1006547238,2023-12-28 16:00:00,2023-12-28 16:10:00,e-scooter,0 days 00:10:00,0.849810,849.809943,2304.0,Bonn Alte Schulstr.,801621 POLYGON ((7.157885983913106 50.7227...,NaN,None,801621 ...,POINT (7.15428 50.74919),POINT (7.15354 50.74152)


In [13]:
#vrs_data = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/11_data_clean.parquet", engine="pyarrow"))
vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/10_VRS_clean.csv"))

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1730/1289783509.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/11_data_clean.csv"))


In [14]:
# transform geometry
vrs_data['station_point'] = vrs_data['station_point'].apply(wkt.loads)
vrs_data['buffer_zone'] = shapely.wkt.loads(vrs_data['buffer_zone'])
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='buffer_zone')
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='station_point')

In [15]:
# transform geometry
#point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["station_point"], crs=4326))
#buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["buffer_zone"], crs=4326))

In [16]:
# drop geometry columns in wrong format
#vrs_data= vrs_data.drop(columns=["station_point", "buffer_zone"])

In [17]:
# replace geometry columns with correct format
#vrs_data['station_point'] = point
#vrs_data['buffer_zone'] = buffer

# TO DO: Match Start times

## Match Tier Data with VRS Data

### A: Count entries
A1: Amount of trips starting within timeframe of x minutes
A2: Amount of those trips, that ended at another stations


In [18]:
#tier_trips_start_time
#scheduled_departure_time

In [19]:
vrs = vrs_data.copy()

In [20]:
# convert to datetime
vrs['actual_arrival_time'] =  pd.to_datetime(vrs['actual_arrival_time'])
vrs['actual_departure_time'] =  pd.to_datetime(vrs['actual_departure_time'])
vrs['scheduled_arrival_time'] =  pd.to_datetime(vrs['scheduled_arrival_time'])
vrs['scheduled_departure_time'] =  pd.to_datetime(vrs['scheduled_departure_time'])

In [21]:
# keep only tier data where a trip staerted at a station
tier_trips_not_nan = tier_trips[tier_trips['start_stop_id'].notna()]

In [22]:
# Slice the dataframe into three equal parts for easier processing
num_rows = len(vrs)
part_size = num_rows // 3

# First part
part1 = vrs.iloc[:part_size]

# Second part
part2 = vrs.iloc[part_size:2*part_size]

# Third part
part3 = vrs.iloc[2*part_size:]

In [23]:
# Create a new column in vrs_data to store the count
part1['tier_trips_count'] = 0
part1['tier_trips_id'] = None
part1['tier_trips_end_at_station_count'] = 0
part1['tier_trips_end_at_station_id'] = None

# Iterate over each row in vrs_data
for index, row in part1.iterrows():
    # Get the stop_id and scheduled_arrival_time
    stop_id = row['stop_id']
    scheduled_departure_time = row['scheduled_departure_time']
    
    # Calculate the time span
    time_span = pd.Timedelta(minutes=5)
    
    # Filter tier_trips based on stop_id and time span
    filtered_trips = tier_trips_not_nan[(tier_trips_not_nan['start_stop_id'] == stop_id) & 
                                (tier_trips_not_nan['tier_trips_start_time'] >= scheduled_departure_time) & 
                                (tier_trips_not_nan['tier_trips_start_time'] <= scheduled_departure_time + time_span)]
    
    filtered_ids = filtered_trips['tier_trips_id'].values
    
    # Count the number of entries
    count = len(filtered_ids)
    
    # Update the count in vrs_data
    part1.at[index, 'tier_trips_count'] = count
    part1.at[index, 'tier_trips_id'] = filtered_ids # list of tier_trips ids

    # Trips that ended at station
    end_trips = filtered_trips[filtered_trips['end_stop_id'].notna()]
    end_trips_ids = end_trips['tier_trips_id'].values
    end_trips_count = len(end_trips_ids)

    part1.at[index, 'tier_trips_end_at_station_count'] = end_trips_count
    part1.at[index, 'tier_trips_end_at_station_id'] = end_trips_ids

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [24]:
# create a GeoDataFrame
gdf1 = gpd.GeoDataFrame(part1, geometry='station_point')

In [25]:
#
#gdf1 = gdf1[gdf1['tier_trips_count'] != 0]

In [26]:
# convert to string
gdf1['tier_trips_id'] = gdf1['tier_trips_id'].astype(str)

In [27]:
# remove brackets
gdf1['tier_trips_id'] = gdf1['tier_trips_id'].str.replace('[', '').str.replace(']', '')

In [28]:
# convert to string
gdf1['tier_trips_end_at_station_id'] = gdf1['tier_trips_end_at_station_id'].astype(str)
gdf1['route_short_name'] = gdf1['route_short_name'].astype(str)

In [29]:
# export to parquet
gdf1.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/13_VRS_Tier_Merge_1_NEU_NEU_TEST.parquet", engine="pyarrow")

In [30]:
# Create a new column in vrs_data to store the count
part2['tier_trips_count'] = 0
part2['tier_trips_id'] = None
part2['tier_trips_end_at_station_count'] = 0
part2['tier_trips_end_at_station_id'] = None

# Iterate over each row in vrs_data
for index, row in part2.iterrows():
    # Get the stop_id and scheduled_arrival_time
    stop_id = row['stop_id']
    scheduled_arrival_time = row['scheduled_departure_time']
    
    # Calculate the time span
    time_span = pd.Timedelta(minutes=5)
    
    # Filter tier_trips based on stop_id and time span
    filtered_trips = tier_trips_not_nan[(tier_trips_not_nan['start_stop_id'] == stop_id) & 
                                (tier_trips_not_nan['tier_trips_start_time'] >= scheduled_arrival_time) & 
                                (tier_trips_not_nan['tier_trips_start_time'] <= scheduled_arrival_time + time_span)]
    
    filtered_ids = filtered_trips['tier_trips_id'].values
    
    # Count the number of entries
    count = len(filtered_ids)
    
    # Update the count in vrs_data
    part2.at[index, 'tier_trips_count'] = count
    part2.at[index, 'tier_trips_id'] = filtered_ids # list of tier_trips ids

    # Trips that ended at station
    end_trips = filtered_trips[filtered_trips['end_stop_id'].notna()]
    end_trips_ids = end_trips['tier_trips_id'].values
    end_trips_count = len(end_trips_ids)

    part2.at[index, 'tier_trips_end_at_station_count'] = end_trips_count
    part2.at[index, 'tier_trips_end_at_station_id'] = end_trips_ids

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [31]:
gdf2 = gpd.GeoDataFrame(part2, geometry='station_point')

In [32]:
#gdf2 = gdf2[gdf2['tier_trips_count'] != 0]

In [33]:
gdf2['tier_trips_id'] = gdf2['tier_trips_id'].astype(str)

In [34]:
gdf2['tier_trips_id'] = gdf2['tier_trips_id'].str.replace('[', '').str.replace(']', '')

In [35]:
gdf2['tier_trips_end_at_station_id'] = gdf2['tier_trips_end_at_station_id'].astype(str)
gdf2['route_short_name'] = gdf2['route_short_name'].astype(str)
gdf2.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/13_VRS_Tier_Merge_2_NEU_NEU_TEST.parquet", engine="pyarrow")

In [36]:
# Create a new column in vrs_data to store the count
part3['tier_trips_count'] = 0
part3['tier_trips_id'] = None
part3['tier_trips_end_at_station_count'] = 0
part3['tier_trips_end_at_station_id'] = None

# Iterate over each row in vrs_data
for index, row in part3.iterrows():
    # Get the stop_id and scheduled_arrival_time
    stop_id = row['stop_id']
    scheduled_departure_time = row['scheduled_departure_time']
    
    # Calculate the time span
    time_span = pd.Timedelta(minutes=5)
    
    # Filter tier_trips based on stop_id and time span
    filtered_trips = tier_trips_not_nan[(tier_trips_not_nan['start_stop_id'] == stop_id) & 
                                (tier_trips_not_nan['tier_trips_start_time'] >= scheduled_departure_time) & 
                                (tier_trips_not_nan['tier_trips_start_time'] <= scheduled_departure_time + time_span)]
    
    filtered_ids = filtered_trips['tier_trips_id'].values
    
    # Count the number of entries
    count = len(filtered_ids)
    
    # Update the count in vrs_data
    part3.at[index, 'tier_trips_count'] = count
    part3.at[index, 'tier_trips_id'] = filtered_ids # list of tier_trips ids

    # Trips that ended at station
    end_trips = filtered_trips[filtered_trips['end_stop_id'].notna()]
    end_trips_ids = end_trips['tier_trips_id'].values
    end_trips_count = len(end_trips_ids)

    part3.at[index, 'tier_trips_end_at_station_count'] = end_trips_count
    part3.at[index, 'tier_trips_end_at_station_id'] = end_trips_ids

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [37]:
gdf3 = gpd.GeoDataFrame(part3, geometry='station_point')

In [38]:
#gdf3 = gdf3[gdf3['tier_trips_count'] != 0]

In [39]:
gdf3['tier_trips_id'] = gdf3['tier_trips_id'].astype(str)

In [40]:
gdf3['tier_trips_id'] = gdf3['tier_trips_id'].str.replace('[', '').str.replace(']', '')

In [41]:
gdf3['tier_trips_end_at_station_id'] = gdf3['tier_trips_end_at_station_id'].astype(str)
gdf3['route_short_name'] = gdf3['route_short_name'].astype(str)
gdf3.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/13_VRS_Tier_Merge_3_NEU_NEU_TEST.parquet", engine="pyarrow")

In [42]:
# combine the three GeoDataFrames
combined = pd.concat([gdf1, gdf2, gdf3])

In [43]:
# export to parquet
combined.to_parquet("../Data/13_VRS_Tier_Merge_Complete.parquet", engine="pyarrow")

In [44]:
combined

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,...,arrival_delay,departure_delay,service,date,weekday,transfer_stop,tier_trips_count,tier_trips_id,tier_trips_end_at_station_count,tier_trips_end_at_station_id
0,93805,93805,93805,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,...,NaN,0 days,1,2023-07-03,Monday,True,0,,0,[]
1,104587,104587,104587,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,...,NaN,0 days,1,2023-07-01,Saturday,False,0,,0,[]
2,104692,104692,104692,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,...,NaN,0 days,1,2023-07-02,Sunday,False,0,,0,[]
3,104588,104588,104588,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,...,0 days,0 days,1,2023-07-01,Saturday,False,0,,0,[]
4,104693,104693,104693,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,...,0 days,0 days,1,2023-07-02,Sunday,False,0,,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,115985,115985,115985,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,NaN,NaN,1,2023-07-02,Sunday,False,0,,0,[]
873987,115984,115984,115984,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,NaN,NaN,1,2023-07-02,Sunday,True,0,,0,[]
873988,115986,115986,115986,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,NaN,NaN,1,2023-07-02,Sunday,True,0,,0,[]
873989,115987,115987,115987,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,...,NaN,NaN,1,2023-07-02,Sunday,True,0,,0,[]


In [45]:
tier_trips.columns

Index(['tier_trips_id', 'tier_trips_start_time', 'tier_trips_end_time',
       'vehicle type', 'trip_duration', 'trip_distance_kilometers',
       'trip_distance_meters', 'start_stop_id', 'start_stop_name',
       'start_buffer_zone', 'end_stop_id', 'end_stop_name', 'end_buffer_zone',
       'start_location', 'end_location'],
      dtype='object')

In [46]:
entries = tier_trips[(tier_trips['start_stop_id'].notna()) & (tier_trips['end_stop_id'].notna()) & (tier_trips['start_stop_id'] != tier_trips['end_stop_id'])]
entries

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,start_location,end_location
801667,497085196,2023-01-05 20:20:00,2023-01-05 20:35:00,e-scooter,0 days 00:15:00,2.056873,2056.873148,1500.0,Bonn Konrad-Adenauer-Platz,801621 POLYGON ((7.157885983913106 50.7227...,1128.0,Bonn Gartenstr.,801621 ...,POINT (7.11783 50.73927),POINT (7.13607 50.74233)
801688,497085217,2023-01-06 18:00:00,2023-01-06 18:15:00,e-scooter,0 days 00:15:00,3.244179,3244.179137,1500.0,Bonn Konrad-Adenauer-Platz,801621 POLYGON ((7.157885983913106 50.7227...,6992.0,Bonn Sachsenweg,801621 ...,POINT (7.11705 50.73960),POINT (7.08803 50.74261)
801734,497085263,2023-01-11 15:50:00,2023-01-11 16:00:00,e-scooter,0 days 00:10:00,1.681363,1681.362514,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,688.0,Bonn West,801621 ...,POINT (7.09680 50.73581),POINT (7.08183 50.73796)
801843,497085398,2023-01-07 16:10:00,2023-01-07 16:20:00,e-scooter,0 days 00:10:00,2.017558,2017.558172,1384.0,Bonn Schloßplatz,801621 POLYGON ((7.157885983913106 50.7227...,1380.0,Bonn Reichsstr.,801621 ...,POINT (7.06900 50.67699),POINT (7.06889 50.69527)
801854,497085409,2023-01-09 05:50:00,2023-01-09 06:05:00,e-scooter,0 days 00:15:00,1.460087,1460.086602,1380.0,Bonn Reichsstr.,801621 POLYGON ((7.157885983913106 50.7227...,1313.0,Bonn Lengsdorf Friedhof,801621 ...,POINT (7.06927 50.69342),POINT (7.06835 50.70662)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807512,1006547153,2023-12-29 08:40:00,2023-12-29 09:00:00,e-scooter,0 days 00:20:00,1.413906,1413.906265,1181.0,Bonn Am Nordpark,801621 POLYGON ((7.157885983913106 50.7227...,2738.0,Bonn Hedwigschule,801621 ...,POINT (7.08730 50.74444),POINT (7.07634 50.75094)
4807519,1006547160,2023-12-29 18:05:00,2023-12-29 18:10:00,e-scooter,0 days 00:05:00,1.272025,1272.025410,2738.0,Bonn Hedwigschule,801621 POLYGON ((7.157885983913106 50.7227...,1181.0,Bonn Am Nordpark,801621 ...,POINT (7.07629 50.75100),POINT (7.08615 50.74515)
4807525,1006547166,2023-12-30 08:30:00,2023-12-30 08:40:00,e-scooter,0 days 00:10:00,1.498038,1498.038059,1181.0,Bonn Am Nordpark,801621 POLYGON ((7.157885983913106 50.7227...,1102.0,Bonn Friedensplatz,801621 ...,POINT (7.08621 50.74504),POINT (7.09776 50.73805)
4807532,1006547173,2023-12-30 16:30:00,2023-12-30 16:35:00,e-scooter,0 days 00:05:00,1.088905,1088.904507,1102.0,Bonn Friedensplatz,801621 POLYGON ((7.157885983913106 50.7227...,1153.0,Bonn Heerstr.,801621 ...,POINT (7.09806 50.73763),POINT (7.08827 50.73740)


# REWORK WITH TIER DATASET - AKTUELL EIN FEHLER - ALLE STARTEN UND ENDEN AN STATION

In [47]:
entries = combined[(combined['tier_trips_count'] > 0) & (combined['tier_trips_end_at_station_count'] > 0) & (combined['tier_trips_count'] != combined['tier_trips_end_at_station_count'])]
entries

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,...,arrival_delay,departure_delay,service,date,weekday,transfer_stop,tier_trips_count,tier_trips_id,tier_trips_end_at_station_count,tier_trips_end_at_station_id
327,117873,117873,117873,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11620,...,NaN,0 days,1,2023-07-01,Saturday,False,2,782275130 782438845,1,782275130
898,100282,100282,100282,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26792,...,NaN,0 days 00:00:00,1,2023-07-02,Sunday,False,2,782313893 782438852,1,782313893
1611,132520,132520,132520,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11621,...,NaN,0 days,1,2023-07-03,Monday,True,2,782271631 782313206,1,782313206
2526,134962,134962,134962,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11621,...,NaN,NaN,1,2023-07-01,Saturday,False,2,782418812 782429972,1,782429972
2881,124276,124276,124276,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11621,...,NaN,NaN,1,2023-07-01,Saturday,True,2,782381108 782417778,1,782417778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871942,5833298,5833298,5833298,68,6,68,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:00,0 days 00:00:00,1,2023-07-12,Wednesday,False,2,782414648 782457434,1,782457434
871944,5833943,5833943,5833943,68,6,68,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:00,0 days 00:00:00,1,2023-07-14,Friday,False,3,808702671 808802704 808632305,2,[808702671 808802704]
871946,5835818,5835818,5835818,68,6,68,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:00,0 days 00:00:00,1,2023-07-18,Tuesday,False,3,808672025 808804918 808626762,1,808626762
872354,5839388,5839388,5839388,68,6,68,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:00,0 days 00:00:00,1,2023-07-06,Thursday,False,2,782320393 782354341,1,782354341
